# Self-supervised and multi-modal representation Learning: Notebook 3

Here we will align the image and light curve representations with contrastive learning, using CLIP (https://openai.com/research/clip). 

- From https://openai.com/research/clip.

## Aligning light curves and galaxy images in a common embedding space

In [3]:
import os, sys

import numpy as np
import pandas as pd
import math
from typing import Dict, Optional
import matplotlib.pyplot as plt

from tqdm import tqdm
from PIL import Image
from einops import rearrange

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import TensorDataset, DataLoader, random_split
from torchvision.transforms import RandomRotation
from torch.utils.data import TensorDataset, DataLoader, random_split
from src.models_multimodal import (
    ConvMixer,
    TransformerWithTimeEmbeddings,
    LightCurveImageCLIP,
)
from src.utils import (
    get_valid_dir,
    LossTrackingCallback,
    plot_loss_history,
    get_embs,
    find_indices_in_arrays,
    get_AUC,
    LossTrackingCallback,
)
from src.dataloader import (
    load_images,
    load_lightcurves,
    plot_lightcurve_and_images,
    load_spectras,
    load_data,
    NoisyDataLoader,
)
from src.loss import sigmoid_loss, clip_loss
from typing import Tuple
from src.transformer_utils import Transformer

from IPython.display import Image as IPImage

%load_ext autoreload
%autoreload 2

### Data preprocessing

Load host images and inspect shape.

In [4]:
max_data_len = 1000  # Spectral data is cut to this length
dataset, nband = load_data("ZTFBTS/", "ZTFBTS_spectra/", max_data_len, True)

number_of_samples = len(dataset)

val_fraction = 0.05
batch_size = 1
n_samples_val = int(val_fraction * number_of_samples)

dataset_train, dataset_val = random_split(
    dataset, [number_of_samples - n_samples_val, n_samples_val]
)

train_loader_no_aug = DataLoader(
    dataset_train, batch_size=batch_size, num_workers=1, pin_memory=True, shuffle=True
)
val_loader_no_aug = DataLoader(
    dataset_val, batch_size=batch_size, num_workers=1, pin_memory=True, shuffle=False
)

spectral and host galaxy
Loading images...


100%|█████████████████████████████████████████████████████████████████████████████| 5171/5171 [00:04<00:00, 1164.39it/s]


Loading spectra ...


100%|██████████████████████████████████████████████████████████████████████████████| 5095/5095 [00:20<00:00, 250.58it/s]


### Create dataset

In [5]:
dataset_train, dataset_val = random_split(
    dataset, [number_of_samples - n_samples_val, n_samples_val]
)
train_loader_no_aug = DataLoader(
    dataset_train, batch_size=batch_size, num_workers=8, pin_memory=True, shuffle=True
)
val_loader_no_aug = DataLoader(
    dataset_val, batch_size=batch_size, num_workers=8, pin_memory=True, shuffle=False
)

# Define the noise levels for images and magnitude (multiplied by magerr)
noise_level_img = 1  # Adjust as needed
noise_level_mag = 1  # Adjust as needed

val_noise = 0

# Create custom noisy data loaders
train_loader = NoisyDataLoader(
    dataset_train,
    batch_size=batch_size,
    noise_level_img=noise_level_img,
    noise_level_mag=noise_level_mag,
    shuffle=True,
    num_workers=8,
    pin_memory=True,
)
val_loader = NoisyDataLoader(
    dataset_val,
    batch_size=batch_size,
    noise_level_img=val_noise,
    noise_level_mag=val_noise,
    shuffle=False,
    num_workers=8,
    pin_memory=True,
)

### Contrastive-style losses

The standard CLIP architecture uses a bidirection (symmetric between modalities, e.g. image and text) version of the so-called SimCLR loss to compute alignment between image and light curve representations.
$$\mathcal{L}_\mathrm{CLIP}=-\frac{1}{2|\mathcal{B}|} \sum_{i=1}^{|\mathcal{B}|}\left(\log \frac{e^{t\,x_i \cdot y_i}}{\sum_{j=1}^{|\mathcal{B}|} e^{t\,x_i \cdot y_j}}+\log \frac{e^{t\,x_i \cdot y_i}}{\sum_{j=1}^{|\mathcal{B}|} e^{t\,x_j \cdot y_i}}\right)$$

The standard CLIP loss can be quite unstable due to the small number of positive pairs and large number of negative pairs in a batch. It can also often require very large batch sizes to work well. There are many proposed ways of overcoming this, e.g. see https://lilianweng.github.io/posts/2021-05-31-contrastive/ for some approaches. 

In addition to theh softmax-based loss, we'll also try a sigmoid loss, from https://arxiv.org/abs/2303.15343:
$$\mathcal{L}_\mathrm{SigLIP}=-\frac{1}{|\mathcal{B}|} \sum_{i=1}^{|\mathcal{B}|} \sum_{j=1}^{|\mathcal{B}|} \log \frac{1}{1+e^{z_{i j}\left(-t\, {x}_i \cdot {y}_j+b\right)}}$$
where $x_i$ and $y_j$ are the normalized image and light curve representations, respectively, and $z_{ij}$ is a binary indicator of whether the image and light curve are a match or not.

Let's implement these two.

In [7]:
transformer_kwargs = {"n_out": 32, "emb": 32, "heads": 2, "depth": 1, "dropout": 0.0}
conv_kwargs = {
    "dim": 32,
    "depth": 2,
    "channels": 3,
    "kernel_size": 5,
    "patch_size": 10,
    "n_out": 32,
    "dropout_prob": 0.0,
}
optimizer_kwargs = {"weight_decay": 0.0}

clip_model = LightCurveImageCLIP(
    logit_scale=20.0,
    lr=1e-4,
    loss="softmax",
    transformer_kwargs=transformer_kwargs,
    conv_kwargs=conv_kwargs,
)

In [ ]:
loss_tracking_callback = LossTrackingCallback()
trainer = pl.Trainer(
    max_epochs=50, accelerator="gpu", callbacks=[loss_tracking_callback]
)
trainer.fit(
    model=clip_model, train_dataloaders=train_loader, val_dataloaders=val_loader
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/tom/Desktop/temp_folder_trasnsfer/Multimodal-hackathon-2024/dev/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name                  | Type                          | Params
------------------------------------------------------------------------
0 | lightcurve_encoder    | TransformerWithTimeEmbeddings | 13.7 K
1 | image_encoder         | ConvMixer                     | 80

Sanity Checking: |                                                                                | 0/? [00:00…

/Users/tom/Desktop/temp_folder_trasnsfer/Multimodal-hackathon-2024/dev/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
/Users/tom/Desktop/temp_folder_trasnsfer/Multimodal-hackathon-2024/src/utils.py:194: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3641.)
  logits = a_norm @ b_norm.T * temperature
/Users/tom/Desktop/temp_folder_trasnsfer/Multimodal-hackathon-2024/dev/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=Tr

Training: |                                                                                       | 0/? [00:00…

Play around with various hyperparameters. Does the loss go down/converge? **Is there actually a strong correlation between the host images and light curves?** How can you test this? (E.g. -> overfit on a small batch).

In [55]:
train_loss_history, val_loss_history = (
    loss_tracking_callback.train_loss_history,
    loss_tracking_callback.val_loss_history,
)

# Create a figure and a set of subplots
plt.figure(figsize=(10, 6))

# Plot training loss
plt.plot(
    train_loss_history, label="Training Loss", color="blue", linestyle="-", marker="o"
)

# Plot validation loss
plt.plot(
    val_loss_history, label="Validation Loss", color="red", linestyle="-", marker="x"
)

# Adding title and labels
plt.title("Training and Validation Loss Over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")

# Adding a legend
plt.legend()

# Show grid
plt.grid(True)

## Downstream task: retrieval

Try your hand at various downstream tasks: compute embeddings over images, see if they are able to identify similar images/light curves better than the base model. See if the model has learned useful joint representations by trying retrieval across modalities (image to light curve and vice versa).

Looks like we can pick up similar light curves. Can you improve the model -- the light curves encoder (see last notebook), the image model, or the loss function?

In [58]:
def cosine_similarity(a, b, temperature=1):
    a_norm = a / a.norm(dim=-1, keepdim=True)
    b_norm = b / b.norm(dim=-1, keepdim=True)

    logits = a_norm @ b_norm.T * temperature
    return logits.squeeze()

In [67]:
embs_curves_train, embs_images_train = get_embs(clip_model, train_loader_no_aug)
embs_curves_val, embs_images_val = get_embs(clip_model, val_loader_no_aug)

In [68]:
import matplotlib.pyplot as plt


def plot_ROC_curves(
    embs_curves_train: torch.Tensor,
    embs_images_train: torch.Tensor,
    embs_curves_val: torch.Tensor,
    embs_images_val: torch.Tensor,
) -> None:
    """
    Plots ROC-like curves for training and validation datasets based on embeddings.

    Args:
    embs_curves_train (torch.Tensor): Embeddings for light curves in the training set.
    embs_images_train (torch.Tensor): Embeddings for images in the training set.
    embs_curves_val (torch.Tensor): Embeddings for light curves in the validation set.
    embs_images_val (torch.Tensor): Embeddings for images in the validation set.
    """
    thresholds, fraction_correct_train = get_ROC_data(
        embs_curves_train, embs_images_train
    )
    thresholds, fraction_correct_val = get_ROC_data(embs_curves_val, embs_images_val)

    # Set overall figure size and title
    plt.figure(figsize=(12, 6))
    plt.suptitle("Fraction of Correct Predictions vs. Threshold")

    # Plot for validation data
    plt.subplot(1, 2, 1)
    plt.plot(thresholds, fraction_correct_val, color="blue", lw=2, label="Validation")
    plt.plot(thresholds, thresholds, linestyle="--", color="gray", label="Random")
    plt.title("Validation Data")
    plt.xlabel("Threshold")
    plt.ylabel("Fraction Correct")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.7)

    # Plot for training data
    plt.subplot(1, 2, 2)
    plt.plot(thresholds, fraction_correct_train, color="green", lw=2, label="Training")
    plt.plot(thresholds, thresholds, linestyle="--", color="gray", label="Random")
    plt.title("Training Data")
    plt.xlabel("Threshold")
    plt.ylabel("Fraction Correct")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.7)

    # Adjust layout to prevent overlapping
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


plot_ROC_curves(embs_curves_train, embs_images_train, embs_curves_val, embs_images_val)

### Downstream tasks: detecting interesting samples from a big survey sample

Since we have access to a huge collection of (bulk) light curves (see `xx_ssl_lightcurve_encoder.ipynb`), we can also try to find interesting samples from this super noisy dataset based on the learned multimodal representations, by asking which light curves from the bulk samples are similar to given "interesting" light curves from the BTS sample.

### Downstream tasks: fine-tuning and classification

Since we also have access to properties like redshift, we can train a regression head on top of the learned representations, to then classify the bulk light curves.

In [ ]:
df_properties = pd.read_csv("../data/ZTFBTS/ZTFBTS_TransientTable.csv")
df_properties.head()

In particular, try to just do linear regression (add a single `nn.Linear` layer on top) after freezing the learned encoder weights from either image and/or light curve representations to see if you can do inference on e.g. the redshift.